# Extract Phase

In this step, we perform the Extract phase of the ETL process.

**Objectives:**
- Load raw and incremental datasets.
- Inspect the datasets to understand structure and contents.
- Check for data quality issues (duplicates, outliers, missing values, wrong types).
- Combine raw and incremental datasets to create a validated dataset for further processing.


In [15]:
# Import pandas for data manipulation
import pandas as pd


## Load Raw and Incremental Datasets

We load the raw and incremental CSV files into pandas DataFrames.


In [16]:
# Load the raw dataset
raw_df = pd.read_csv('data/raw_data.csv')

# Load the incremental dataset
inc_df = pd.read_csv('data/incremental_data.csv')

# Rename columns for easier access
raw_df.columns = ['Year', 'Cause_Code', 'Cause_Name', 'State', 'Deaths', 'Age_Adjusted_Rate']
inc_df.columns = ['Year', 'Cause_Code', 'Cause_Name', 'State', 'Deaths', 'Age_Adjusted_Rate']


## Inspecting Raw Data

We inspect the raw dataset using `head()`, `info()`, and `describe()` to understand its structure, columns, and summary statistics.


In [17]:
# Display first 5 rows
raw_df.head()

# Check column info, data types, and missing values
raw_df.info()

# Summary statistics for numeric columns
raw_df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10868 entries, 0 to 10867
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Year               10868 non-null  int64  
 1   Cause_Code         10868 non-null  object 
 2   Cause_Name         10868 non-null  object 
 3   State              10868 non-null  object 
 4   Deaths             10868 non-null  int64  
 5   Age_Adjusted_Rate  10868 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 509.6+ KB


,Year,Deaths,Age_Adjusted_Rate
count,10868.000000,1.086800e+04,10868.000000
mean,2008.000000,1.545991e+04,127.563894
std,5.477478,1.128760e+05,223.639771
min,1999.000000,2.100000e+01,2.600000
25%,2003.000000,6.120000e+02,19.200000
50%,2008.000000,1.718500e+03,35.900000
75%,2013.000000,5.756500e+03,151.725000
max,2017.000000,2.813503e+06,1087.300000


## Data Quality Observations

After inspecting the raw dataset, we identified the following issues:

1. **Duplicate rows:** None were found, but we will still check in the transform phase.
2. **Potential outliers:** Some values in 'Deaths' and 'Age_Adjusted_Rate' are unusually high or low.
3. **Cause Name consistency:** 'Cause_Name' capitalization is consistent, but the 'Cause_Code' column is missing codes, which may affect analysis.

**Action:** These issues will be addressed in the Transform phase.


In [18]:
# Check for duplicate rows
duplicate_rows = raw_df[raw_df.duplicated()]
print(f"Number of duplicate rows in raw dataset: {duplicate_rows.shape[0]}")
print("Sample duplicate rows:")
print(duplicate_rows.head())

# Detect potential outliers using IQR method for Deaths
Q1 = raw_df['Deaths'].quantile(0.25)
Q3 = raw_df['Deaths'].quantile(0.75)
IQR = Q3 - Q1

outliers_deaths = raw_df[(raw_df['Deaths'] < (Q1 - 1.5*IQR)) | (raw_df['Deaths'] > (Q3 + 1.5*IQR))]
print(f"\nNumber of potential outliers in 'Deaths': {outliers_deaths.shape[0]}")
print(outliers_deaths.head())

# Detect potential outliers for Age_Adjusted_Rate
Q1_rate = raw_df['Age_Adjusted_Rate'].quantile(0.25)
Q3_rate = raw_df['Age_Adjusted_Rate'].quantile(0.75)
IQR_rate = Q3_rate - Q1_rate

outliers_rate = raw_df[(raw_df['Age_Adjusted_Rate'] < (Q1_rate - 1.5*IQR_rate)) | 
                       (raw_df['Age_Adjusted_Rate'] > (Q3_rate + 1.5*IQR_rate))]
print(f"\nNumber of potential outliers in 'Age_Adjusted_Rate': {outliers_rate.shape[0]}")
print(outliers_rate.head())


Number of duplicate rows in raw dataset: 0
Sample duplicate rows:
Empty DataFrame
Columns: [Year, Cause_Code, Cause_Name, State, Deaths, Age_Adjusted_Rate]
Index: []

Number of potential outliers in 'Deaths': 1483
    Year                                         Cause_Code  \
0   2017  Accidents (unintentional injuries) (V01-X59,Y8...   
5   2017  Accidents (unintentional injuries) (V01-X59,Y8...   
52  2017                                         All Causes   
53  2017                                         All Causes   
55  2017                                         All Causes   

                Cause_Name          State   Deaths  Age_Adjusted_Rate  
0   Unintentional injuries  United States   169936               49.4  
5   Unintentional injuries     California    13840               33.2  
52              All causes  United States  2813503              731.9  
53              All causes        Alabama    53238              917.7  
55              All causes        Arizona    57

## Combining Raw and Incremental Data

We append the incremental dataset to the raw dataset using `pd.concat()`.

**Reason:**
- The incremental dataset contains recent entries not present in the raw dataset.
- Combining ensures we have a single, validated dataset for further analysis and transformation.


In [19]:
# Combine raw and incremental datasets
combined_df = pd.concat([raw_df, inc_df], ignore_index=True)
print(f"Combined dataset shape: {combined_df.shape}")

# Save the validated dataset inside 'data/' folder
combined_df.to_csv('data/validated_data.csv', index=False)
print("Validated dataset saved as 'data/validated_data.csv' in the data folder")


Combined dataset shape: (11868, 6)
Validated dataset saved as 'data/validated_data.csv' in the data folder
